# Trabalho Prático 2 
## Music Retrieval Information 

- Joana Simões, Nº 2019217013
- Samuel Carinhas, Nº2019217199
- Sofia Alves, Nº 2019227240

Este trabalho teve como objetivo desenvolver um sistema de recomendações de músicas com base no conteúdo. Para tal, foram seguidos vários passos essenciais que foram desde a extração de features das músicas presentes no *dataset*, passando posteriormente à seleção das melhores recomendações consoante diferentes métricas para determinadas *queries* usadas como *input*. Por último, houve uma etapa de avaliação subjetiva individual feita por cada membro para calcular a precisão do sistema de recomendação. 

In [1]:
# imports
import librosa
import librosa.display
import sounddevice as sd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.stats as st
from sklearn import preprocessing
import pandas as pd

## Extração de features

In [45]:
def read_features(fileName, delim=','):
    return np.genfromtxt(fileName, delimiter=delim)

In [46]:
def clean_data(data):
    nl, nc = data.shape
    return data[1:, 1:(nc-1)]

In [47]:
def normalize(feature):
    return preprocessing.minmax_scale(feature, feature_range=(0, 1))

In [48]:
def normalize_features(features):
    return np.apply_along_axis(normalize, 0, features) # 0 = columns

In [49]:
top100 = read_features('./dataset/top100_features.csv')
top100 = clean_data(top100)
top100_normalized = normalize_features(top100)

np.savetxt('./dataset/top100_features_normalized.csv', top100_normalized, delimiter=',')

In [18]:
def get_musics(dir='./dataset/musics/'):
    files = os.listdir(dir)
    #print(f'{len(files)} musics found')
    return sorted(files)

In [51]:
def get_statistics(feature):
    if len(feature.shape) == 1:
        feature = feature.reshape(1, feature.shape[0])

    nl = feature.shape[0]
    feature_statistics = np.zeros((nl, 7))

    for i in range(nl):
        mean = feature[i, :].mean()
        std = feature[i, :].std()
        skew = st.skew(feature[i, :])
        kurtosis = st.kurtosis(feature[i, :])
        median = np.median(feature[i, :])
        mx = feature[i, :].max()
        mn = feature[i, :].min()
        feature_statistics[i, :] = np.array([mean, std, skew, kurtosis, median, mx, mn]) 
    

    return feature_statistics.flatten()

In [52]:
def extract_features(audioName):
    sample_rate = 22050
    use_mono = True
    warnings.filterwarnings('ignore')
    f0_min_freq = 20
    f0_max_freq = sample_rate//2
    mfcc_dim = 13

    music, _ = librosa.load(audioName, sr=sample_rate, mono=use_mono)

    mfcc = librosa.feature.mfcc(music, n_mfcc=mfcc_dim)
    sp_centroid = librosa.feature.spectral_centroid(music)
    sp_bandwidth = librosa.feature.spectral_bandwidth(music)
    sp_contrast = librosa.feature.spectral_contrast(music, n_bands=6)
    sp_flatness = librosa.feature.spectral_flatness(music)
    sp_rolloff = librosa.feature.spectral_rolloff(music)    
    f0 = librosa.yin(music, fmin=f0_min_freq, fmax=f0_max_freq)
    f0[f0 == f0_max_freq] = 0
    rms = librosa.feature.rms(music)
    zcr = librosa.feature.zero_crossing_rate(music)
    tempo = librosa.beat.tempo(music)
    
    features = [mfcc, sp_centroid, sp_bandwidth, sp_contrast, sp_flatness, sp_rolloff, f0, rms, zcr]

    statistics = np.zeros(190)
    prev = 0
    for f in features:
        stats = get_statistics(f)
        #print(stats, len(stats))
        statistics[prev:prev+len(stats)] = stats
        prev += len(stats)

    statistics[-1] = tempo
    return statistics.reshape(1, 190)

    

In [53]:
musics = get_musics()

In [54]:
"""extract_features('./dataset/musics/MT0000004637.mp3')
musics = get_musics()
features = np.zeros((900, 190))
for i, music in enumerate (musics):
    features[i, :] = extract_features(f'./dataset/musics/{music}')
""" 

"extract_features('./dataset/musics/MT0000004637.mp3')\nmusics = get_musics()\nfeatures = np.zeros((900, 190))\nfor i, music in enumerate (musics):\n    features[i, :] = extract_features(f'./dataset/musics/{music}')\n"

In [55]:
# normalized_features = normalize_features(features)

In [56]:
"""np.savetxt('./features/normalized_features.csv', normalized_features, delimiter=',', fmt='%.6f')
np.savetxt('./features/features.csv', features, delimiter=',', fmt='%.6f')"""

"np.savetxt('./features/normalized_features.csv', normalized_features, delimiter=',', fmt='%.6f')\nnp.savetxt('./features/features.csv', features, delimiter=',', fmt='%.6f')"

In [57]:
normalized_features = read_features('./features/normalized_features.csv')

## Métricas de similaridade

In [58]:
def euclidean_distance(featuresA, featuresB):
    return np.sqrt(np.sum(np.square(featuresA-featuresB)))

In [59]:
def manhattan_distance(featuresA, featuresB):
    return np.sum(np.abs(featuresA-featuresB))

In [60]:
def cos_distance(featuresA, featuresB):
    return 1 - (featuresA @ featuresB) / (np.linalg.norm(featuresA) * np.linalg.norm(featuresB))

In [61]:
def calculate_distances(features):
    distance_euclidean = np.zeros((900, 900))
    distance_cos = np.zeros((900, 900))
    distance_manhattan = np.zeros((900, 900))
    for i in range(900):
        for j in range(i+1, 900):
            distance_euclidean[i,j] = distance_euclidean[j,i] = euclidean_distance(features[i, :], features[j,:])
            distance_cos[i,j] =  distance_cos[j,i] = cos_distance(features[i,:], features[j,:])
            distance_manhattan[i,j] = distance_manhattan[j,i] = manhattan_distance(features[i, :], features[j,:])
    return distance_euclidean, distance_cos, distance_manhattan

In [62]:
distance_euclidean, distance_cos, distance_manhattan = calculate_distances(normalized_features)
distance_euclidean100, distance_cos100, distance_manhattan100 = calculate_distances(top100_normalized)

In [63]:
np.savetxt('./features/distances/distance_euclidean.csv', distance_euclidean, delimiter=',', fmt='%.6f')
np.savetxt('./features/distances/distance_euclidean100.csv', distance_euclidean100, delimiter=',', fmt='%.6f')
np.savetxt('./features/distances/distance_cos.csv', distance_cos, delimiter=',', fmt='%.6f')
np.savetxt('./features/distances/distance_cos100.csv', distance_cos100, delimiter=',', fmt='%.6f')
np.savetxt('./features/distances/distance_manhattan.csv', distance_manhattan, delimiter=',', fmt='%.6f')
np.savetxt('./features/distances/distance_manhattan100.csv', distance_manhattan100, delimiter=',', fmt='%.6f')

In [64]:
musics = get_musics()
music_dict = dict((music, i) for i, music in enumerate(musics))

In [65]:
data_q = pd.read_csv("./dataset/panda_dataset_taffc_metadata.csv")

In [66]:
def get_the_closest_musics(music, features_distances, number=20):
    music_index = music_dict[music]
    distances = features_distances[music_index, :]
    sorted_distances = np.argsort(distances)
    print(data_q.loc[music_index]["Quadrant"], end=' ')
    return [(musics[i], data_q.loc[i]["Quadrant"]) for i in sorted_distances[1:number+1]]

In [67]:
print(top100_normalized)

[[0.40833396 0.55892695 0.08767653 ... 0.49339478 0.29677785 0.28512149]
 [0.17434797 0.81639588 0.09864405 ... 0.66779986 0.14344785 0.        ]
 [0.21986539 0.95836272 0.11175995 ... 0.73227772 0.19118833 0.        ]
 ...
 [0.22605274 0.8830899  0.22646862 ... 0.66775423 0.19149261 0.        ]
 [0.33731922 0.82533373 0.28343725 ... 0.6795957  0.26104038 0.53630904]
 [0.38793611 0.80516266 0.21350893 ... 0.54009902 0.13512837 0.26941305]]


In [92]:
for music in get_musics('./Queries/'):
    print('Euclidean:', music, get_the_closest_musics(music, distance_euclidean))
    print('Cos:', music, get_the_closest_musics(music, distance_cos))
    print('Manhattan:', music, get_the_closest_musics(music, distance_manhattan))
    print('Euclidean100:', music, get_the_closest_musics(music, distance_euclidean100))
    print('Cos100:', music, get_the_closest_musics(music, distance_cos100))
    print('Manhattan100:', music, get_the_closest_musics(music, distance_manhattan100))
    print('\n')

Q4 Euclidean: MT0000202045.mp3 [('MT0005129157.mp3', 'Q4'), ('MT0011899302.mp3', 'Q3'), ('MT0012001409.mp3', 'Q2'), ('MT0002233402.mp3', 'Q3'), ('MT0007043504.mp3', 'Q2'), ('MT0007799677.mp3', 'Q2'), ('MT0004428604.mp3', 'Q1'), ('MT0002161109.mp3', 'Q1'), ('MT0011975274.mp3', 'Q2'), ('MT0010624346.mp3', 'Q2'), ('MT0009217411.mp3', 'Q2'), ('MT0000092267.mp3', 'Q4'), ('MT0007766156.mp3', 'Q4'), ('MT0001676671.mp3', 'Q4'), ('MT0005270263.mp3', 'Q3'), ('MT0003787478.mp3', 'Q2'), ('MT0011376343.mp3', 'Q4'), ('MT0005213723.mp3', 'Q2'), ('MT0000732821.mp3', 'Q2'), ('MT0002634024.mp3', 'Q3')]
Q4 Cos: MT0000202045.mp3 [('MT0005129157.mp3', 'Q4'), ('MT0012001409.mp3', 'Q2'), ('MT0011899302.mp3', 'Q3'), ('MT0002233402.mp3', 'Q3'), ('MT0007043504.mp3', 'Q2'), ('MT0007799677.mp3', 'Q2'), ('MT0004428604.mp3', 'Q1'), ('MT0011975274.mp3', 'Q2'), ('MT0002161109.mp3', 'Q1'), ('MT0000092267.mp3', 'Q4'), ('MT0010624346.mp3', 'Q2'), ('MT0009217411.mp3', 'Q2'), ('MT0007766156.mp3', 'Q4'), ('MT0003787478.mp3

### Análise de resultados

Pelos resultados obtidos e analisando as músicas recomendadas para cada *query*, pode-se observar que as músicas retornadas pelas matrizes de distância do *top 100 features* não apresentam tanta similaridade às *queries* como as retornadas pelas matrizes que usam todas as *features*.

Uma possível razão para esta discrepância deve-se ao facto de as *top 100 features* estarem canalizadas para distinguir entre quadrantes de emoções e não para comparar músicas num sistema de recomendações de músicas semelhantes. Outro possível motivo, é o facto de ao usar as 190 *features*, pode estar a haver *overfitting* das músicas pelo que os resultados serão mais precisos nestas músicas. Em comparação, ao usar apenas 100 *features* pode estar a ocorrer *sampling noise* fazendo com que apenas estas features não sejam suficientes para obter as melhores recomendações.


### Tabela com a similaridade das distâncias para as diferentes músicsa em percentagem
| Música | Euclideana - Cosseno | Euclideana - Manhattan | Manhattan - Cosseno |
|-|-|-|-|
|MT0000202045| 90% | 85% | 75% |
|MT0000379144| 80% | 85% | 60% |
|MT0000414517| 15% | 65% | 65% |
|MT0000956340| 85% | 90% | 80% |
|**Média**| **67.50%** | **81.25%** | **70.00%** |

Comparando as músicas comuns retornadas pelas diferentes distâncias, utilizando todas as *features*, verifica-se que a distância Euclidiana e de Manhattan têm uma média de 81.25% de músicas em comum.

## Avaliação objetiva

In [69]:
def get_metadata(file='./dataset/panda_dataset_taffc_metadata.csv'):
    metadata = np.genfromtxt(file, delimiter=',', dtype="str")
    return metadata

In [70]:
metadata_raw = get_metadata()
metadata = metadata_raw[1:, [1, 3, 9, 11]]

In [71]:
def metadata_score(metadata):
    scores = np.zeros((900, 900))
    np.fill_diagonal(scores, -1)
    for i in range(metadata.shape[0]):
        for j in range(i+1, metadata.shape[0]):
            for k in range(metadata.shape[1]):
                listA = metadata[i, k][1:-1].split('; ')
                listB = metadata[j, k][1:-1].split('; ')
                for elem in listB:
                    scores[i, j] = scores[j, i] = scores[i, j] + (1 if elem in listA else 0)
    return scores

In [72]:
score_matrix = metadata_score(metadata)

In [73]:
np.savetxt('./dataset/score_matrix.csv', score_matrix, delimiter=',', fmt='%d')

In [74]:
def get_top_metadata_match(music, score_matrix, top=20):    
    music_index = music_dict[music]
    scores = score_matrix[music_index, :]
    # -scores because np.argsort sorts the array in non decreasing order, but we need a non ascending order, so we can get the -values
    scores_sorted = np.argsort(scores, )[-top:][::-1].astype('int16')
    return [musics[i] for i in scores_sorted[:top]]

In [75]:
for music in get_musics('./Queries/'):
    print('Metadata:', music, get_top_metadata_match(music, score_matrix))
    print('\n'*5)

Metadata: MT0000202045.mp3 ['MT0014475915.mp3', 'MT0012862507.mp3', 'MT0000888329.mp3', 'MT0007556029.mp3', 'MT0031898123.mp3', 'MT0004867564.mp3', 'MT0001494812.mp3', 'MT0003022328.mp3', 'MT0011922905.mp3', 'MT0030369896.mp3', 'MT0007453719.mp3', 'MT0034186620.mp3', 'MT0004850690.mp3', 'MT0011938737.mp3', 'MT0034577404.mp3', 'MT0003025046.mp3', 'MT0005285696.mp3', 'MT0002846256.mp3', 'MT0001058887.mp3', 'MT0007766156.mp3']






Metadata: MT0000379144.mp3 ['MT0031951901.mp3', 'MT0014584473.mp3', 'MT0013080259.mp3', 'MT0013416300.mp3', 'MT0011032905.mp3', 'MT0005157391.mp3', 'MT0005253065.mp3', 'MT0008170600.mp3', 'MT0007652281.mp3', 'MT0007349999.mp3', 'MT0007338724.mp3', 'MT0001526386.mp3', 'MT0004287283.mp3', 'MT0005115042.mp3', 'MT0004131058.mp3', 'MT0001929641.mp3', 'MT0001934726.mp3', 'MT0003863509.mp3', 'MT0029877658.mp3', 'MT0003114552.mp3']






Metadata: MT0000414517.mp3 ['MT0010489498.mp3', 'MT0000040632.mp3', 'MT0010487769.mp3', 'MT0012331779.mp3', 'MT0027048677.mp3', 'MT0

In [76]:
def precision(music, score_matrix, distance, top=20):
    m = get_top_metadata_match(music, score_matrix, top)
    d = get_the_closest_musics(music, distance, top)
    intersect = np.intersect1d(m, d)
    p = len(intersect) / top * 100
    print('%.2f%%' % p, intersect)
    return p, intersect


In [77]:
def metric_precision():
    for music in get_musics('./Queries/'):
        print('MUSIC:', music)
        print('\tEuclidean: ', end='')
        precision(music, score_matrix, distance_euclidean)
        print('\tManhattan: ', end='')
        precision(music, score_matrix, distance_manhattan)
        print('\tCos: ', end='')
        precision(music, score_matrix, distance_cos)
        print('\tEuclidean TOP 100: ', end='')
        precision(music, score_matrix, distance_euclidean100)
        print('\tManhattan TOP 100: ', end='')
        precision(music, score_matrix, distance_manhattan100)
        print('\tCos TOP 100: ', end='')
        precision(music, score_matrix, distance_cos100)

In [78]:
metric_precision()

MUSIC: MT0000202045.mp3
	Euclidean: Q4 5.00% ['MT0007766156.mp3']
	Manhattan: Q4 0.00% []
	Cos: Q4 5.00% ['MT0007766156.mp3']
	Euclidean TOP 100: Q4 0.00% []
	Manhattan TOP 100: Q4 0.00% []
	Cos TOP 100: Q4 0.00% []
MUSIC: MT0000379144.mp3
	Euclidean: Q3 0.00% []
	Manhattan: Q3 0.00% []
	Cos: Q3 0.00% []
	Euclidean TOP 100: Q3 0.00% []
	Manhattan TOP 100: Q3 0.00% []
	Cos TOP 100: Q3 0.00% []
MUSIC: MT0000414517.mp3
	Euclidean: Q1 15.00% ['MT0000040632.mp3' 'MT0003243311.mp3' 'MT0003949060.mp3']
	Manhattan: Q1 10.00% ['MT0000040632.mp3' 'MT0003949060.mp3']
	Cos: Q1 10.00% ['MT0000040632.mp3' 'MT0003949060.mp3']
	Euclidean TOP 100: Q1 5.00% ['MT0012331779.mp3']
	Manhattan TOP 100: Q1 10.00% ['MT0000040632.mp3' 'MT0003949060.mp3']
	Cos TOP 100: Q1 10.00% ['MT0000040632.mp3' 'MT0012331779.mp3']
MUSIC: MT0000956340.mp3
	Euclidean: Q2 0.00% []
	Manhattan: Q2 0.00% []
	Cos: Q2 0.00% []
	Euclidean TOP 100: Q2 15.00% ['MT0002372242.mp3' 'MT0004293364.mp3' 'MT0014615863.mp3']
	Manhattan TOP 100

### Análise de resultados

De um modo geral, as precisões obtidas foram baixas. Possíveis razões encontradas para este resultado, são: (1) uso de parâmetros dos meta dados inadequados; (2) uso de meta dados mal avaliados, isto é, como os erros deste *dataset* provém do erro humano, a avaliação é subjetiva e diferente para cada pessoa; (3) uso de um *dataset* pequeno, o que pode potencializar o erro do *dataset*, referido no ponto anterior, e também levar a um *ranking* incorreto das músicas por não existirem músicas similares suficientes para o total das 20. 

Não foi possível encontrar nenhuma relação entre as diferentes distâncias usadas e/ou entre o número de *features* usadas dado que a precisão variou de música para música.

## Avaliação subjetiva

In [3]:
def add_relevant(df, threshold):
    df['relevant'] = np.where(df['mean'] > threshold, True, False)

In [4]:
def add_mean(df, cols):
    df['mean'] = df[cols].mean(axis=1)

In [5]:
def add_std(df, cols):
    df['std'] = df[cols].std(axis=1)

In [72]:
def calculate_precision(ds):
    return ds.value_counts()[True] / ds.count()

In [77]:
def get_likert_scores():
    warnings.filterwarnings("ignore")
    results = pd.DataFrame(columns=['music', 'precisionTOP', 'precisionMetadata'])
    top_size = 20
    values_metadata = np.zeros((1, 0))
    values_top = np.zeros((1, 0))
    for music in get_musics('./Queries/'):
        df = pd.read_csv(f'./dataset/results/{music}.csv')
        metadata = df[['M1', 'M2', 'M3']].copy()
        top = df[['T1', 'T2', 'T3']].copy()

        values_metadata = np.append(values_metadata, metadata.to_numpy())
        values_top = np.append(values_top, top.to_numpy())

        add_mean(top, ['T1', 'T2', 'T3'])
        add_std(top,['T1', 'T2', 'T3'])
        add_mean(metadata, ['M1', 'M2', 'M3'])
        add_std(metadata,['M1', 'M2', 'M3'])
        threshold = 2.5
        add_relevant(top, threshold)
        add_relevant(metadata, threshold)

        top['music'] = df['music']
        metadata['music'] = df['music']

        top = top.set_index('music')
        metadata = metadata.set_index('music')

        top.to_csv(f'./dataset/results/{music}_top.csv', index=True)
        metadata.to_csv(f'./dataset/results/{music}_metadata.csv', index=True)

        precision_top = calculate_precision(top['relevant'])
        precision_metadata = calculate_precision(metadata['relevant'])

        results = results.append({'music': music, 'precisionTOP': precision_top, 'precisionMetadata': precision_metadata}, ignore_index=True)

        users_results = pd.DataFrame(columns=['User','MeanTop', 'StdTop', 'MeanMetadata', 'StdMetadata'])
        users_results = users_results.append({'User' : '1', 'MeanTop' : df['T1'].mean(), 'StdTop' : df['T1'].std(), 'MeanMetadata' : df['M1'].mean(), 'StdMetadata' : df['M1'].std()}, ignore_index=True)
        users_results = users_results.append({'User' : '2', 'MeanTop' : df['T2'].mean(), 'StdTop' : df['T2'].std(), 'MeanMetadata' : df['M2'].mean(), 'StdMetadata' : df['M2'].std()}, ignore_index=True)
        users_results = users_results.append({'User' : '3', 'MeanTop' : df['T3'].mean(), 'StdTop' : df['T3'].std(), 'MeanMetadata' : df['M3'].mean(), 'StdMetadata' : df['M3'].std()}, ignore_index=True)
        print('Statistics for Likert scale of each student for music:', music)
        print(users_results)
        print('\n')
    return results, values_metadata, values_top

In [78]:
res, values_metadata, values_top = get_likert_scores()
res

Statistics for Likert scale of each student for music: MT0000202045.mp3
  User  MeanTop    StdTop  MeanMetadata  StdMetadata
0    1     1.90  1.071153          3.10     1.165287
1    2     1.85  1.089423          2.45     1.099043
2    3     1.70  0.864505          2.35     1.225819


Statistics for Likert scale of each student for music: MT0000379144.mp3
  User  MeanTop    StdTop  MeanMetadata  StdMetadata
0    1     2.60  1.602629          2.35     1.663066
1    2     2.05  1.316894          1.70     1.174286
2    3     2.30  1.719853          1.45     0.887041


Statistics for Likert scale of each student for music: MT0000414517.mp3
  User  MeanTop    StdTop  MeanMetadata  StdMetadata
0    1     1.75  1.118034          2.15     1.268028
1    2     1.60  1.142481          2.20     1.321881
2    3     1.45  0.998683          1.45     0.998683


Statistics for Likert scale of each student for music: MT0000956340.mp3
  User  MeanTop    StdTop  MeanMetadata  StdMetadata
0    1      3.3  

,music,precisionTOP,precisionMetadata
0,MT0000202045.mp3,0.20,0.40
1,MT0000379144.mp3,0.35,0.25
2,MT0000414517.mp3,0.10,0.25
3,MT0000956340.mp3,0.65,0.55


In [79]:
print("Total recomentations [TOP] mean:", values_top.mean())
print("Total recomentations [TOP] std:", values_top.std())
print("Total recomentations [METADATA] mean:", values_metadata.mean())
print("Total recomentations [METADATA] std:", values_metadata.std())

Total recomentations [TOP] mean: 2.2583333333333333
Total recomentations [TOP] std: 1.437566423586132
Total recomentations [METADATA] mean: 2.3333333333333335
Total recomentations [METADATA] std: 1.359125535858341


In [80]:
print('Likert for music MT0000202045.mp3 by metadata')
print(pd.read_csv('./dataset/results/MT0000202045.mp3_metadata.csv'))
# The other tables can be found in ./dataset/results/{QUERY_NAME}.mp3_metadata.csv
# or ./dataset/results/{QUERY_NAME}.mp3_top.csv

Likert for music MT0000202045.mp3 by metadata
               music  M1  M2  M3      mean       std  relevant
0   MT0033841575.mp3   5   5   5  5.000000  0.000000      True
1   MT0027002641.mp3   3   2   3  2.666667  0.577350      True
2   MT0030487841.mp3   5   4   4  4.333333  0.577350      True
3   MT0008575372.mp3   2   2   1  1.666667  0.577350     False
4   MT0014576739.mp3   3   2   2  2.333333  0.577350     False
5   MT0030422114.mp3   3   1   1  1.666667  1.154701     False
6   MT0003390733.mp3   3   3   1  2.333333  1.154701     False
7   MT0027835071.mp3   2   2   1  1.666667  0.577350     False
8   MT0011145388.mp3   2   3   2  2.333333  0.577350     False
9   MT0009188643.mp3   4   2   3  3.000000  1.000000      True
10  MT0010617945.mp3   2   2   3  2.333333  0.577350     False
11  MT0009213083.mp3   3   1   3  2.333333  1.154701     False
12  MT0005331755.mp3   4   3   2  3.000000  1.000000      True
13  MT0002233402.mp3   4   3   4  3.666667  0.577350      True
14  MT002

### Análise de resultados

Não foi possível calcular o valor do *recall*, que por sua vez impossibilitou o cálculo da *F-measure*, dado que o total de músicas relevantes do *dataset* não é conhecido. Para calcular esta métrica, poderia ser feita uma análise manual de cada música do *dataset*, ou analisar os meta dados e definir um valor *threshold*, a partir do qual podemos considerar a música relevante.

A razão pela qual as precisões obtidas para o *TOP 100 features* foram baixas, deve-se ao facto de a análise subjetiva se basear na similaridade entre as músicas, sendo que as *TOP 100 features* estão canalizadas para distinguir emoções. Já na análise pelos meta dados, teve em conta o estilo, o género e o autor da música, o que levou a uma precisão um pouco melhor. Isto pode dever-se ao facto destes meta dados estarem mais próximos do nosso critério de avaliação. **Estes resultados podem ser observados ao calcular a média e desvio padrão dasmúsicas avaliadas subjetivamente, mostrando que a média das recomendações do *Top 100 features* ser pior do que a que recorre aos meta dados**